## <u>Bước 1</u>: Import các thư viện cần thiết

In [1]:
import datetime as dt
import numpy as np
from abc import abstractmethod
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model

2023-05-06 20:35:21.717516: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## <u>Bước 2</u>: Tải bộ dataset MNIST

In [2]:
# load MNIST dataset
mnist = fetch_openml("mnist_784")

/Users/quytrungg/.pyenv/versions/3.11.2/envs/deep-learning/lib/python3.11/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


## <u>Bước 3</u>: Tiền xử lý dữ liệu & chia dữ liệu thành 2 tập train và test

In [3]:
mnist.data = mnist.data.to_numpy().reshape((-1, 28, 28, 1))
mnist.data.shape

(70000, 28, 28, 1)

In [4]:
# split the dataset into training set and validation set
train_img, test_img, train_lbl, test_lbl = train_test_split(
    mnist.data,
    mnist.target,
    test_size=1/7.0,
    random_state=0,
)

In [5]:
# convert to one-hot vector
train_lbl = to_categorical(train_lbl)
test_lbl = to_categorical(test_lbl)
train_lbl.shape, test_lbl.shape

((60000, 10), (10000, 10))

In [6]:
# check size
train_img.shape, test_img.shape, train_lbl.shape, test_lbl.shape

((60000, 28, 28, 1), (10000, 28, 28, 1), (60000, 10), (10000, 10))

## <u>Bước 4</u>: Xây dựng 3 mô hình sử dụng fully connected và convolutional layer

In [7]:
class AbstractModel:
    """Abstract model architecture class."""

    model = None
    log_dir = "logs/" + dt.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
    )

    @abstractmethod
    def __init__(self, kernel_height: int, kernel_width: int) -> None:
        raise NotImplementedError

    def compile(self, loss: str, optimizer: str, metrics: list[str]) -> None:
        """Compile and summarize initial layers in the model."""
        self.model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        self.model.summary()

    def fit(
        self,
        X_train: np.ndarray,
        y_train: np.ndarray,
        X_test: np.ndarray,
        y_test: np.ndarray,
        epochs: int,
        batch_size: int,
    ) -> None:
        """Train model with training set and evaluate with validation set."""
        self.model.fit(
            X_train,
            y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[self.tensorboard_callback],
        )

    def accuracy(self, X_test: np.ndarray, y_test: np.ndarray) -> float:
        """Return the final model's accuracy."""
        _, accuracy = self.model.evaluate(X_test, y_test)
        return accuracy

    def save_weights(self, filename: str) -> None:
        self.model.save_weights(filename)

    def save(self, filename: str) -> None:
        self.model.save(filename, include_optimizer=False)

### **Mô hình 1 và nhận xét**

In [8]:
class Model1(AbstractModel):
    """Architecture for Model1.

    This model includes:
        - 1 Conv2D layer (50 filters, strides 3x3, ReLU activation)
        - 1 Conv2D layer (50 filters, strides 2x2, ReLU activation)
        - 2 MaxPooling2D layers (size 2x2)
        - 1 Flatten layer
        - 1 Dense layer (256 perceptrons, ReLU activation)
        - 1 Dense layer (128 perceptrons, ReLU activation)
        - 1 Dense layer (64 perceptrons, ReLU activation)
        - 1 Dense layer (10 perceptrons, Softmax activation)

    """

    def __init__(self, kernel_height: int, kernel_width: int) -> None:
        """Initialize layers in the model."""
        self.input_layer = Input(shape=(kernel_height, kernel_width, 1))
        self.conv_layer = Conv2D(50, (3, 3), activation="relu", padding="same")(self.input_layer)
        self.pooling_layer = MaxPooling2D((2, 2))(self.conv_layer)
        self.conv_layer_2 = Conv2D(50, (2, 2), activation="relu", padding="same")(self.pooling_layer)
        self.pooling_layer_2 = MaxPooling2D((2, 2))(self.conv_layer_2)
        self.flatten_layer = Flatten()(self.pooling_layer_2)
        self.dense_layer = Dense(256, activation="relu")(self.flatten_layer)
        self.dense_layer_2 = Dense(128, activation="relu")(self.dense_layer)
        self.dense_layer_3 = Dense(64, activation="relu")(self.dense_layer_2)
        self.output_layer = Dense(10, activation="softmax")(self.dense_layer_3)
        self.model = Model(inputs=self.input_layer, outputs=self.output_layer)

In [9]:
model1 = Model1(28, 28)
model1.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)
model1.fit(train_img, train_lbl, test_img, test_lbl, 5, 2000)
print(f"Model Accuracy: {model1.accuracy(test_img, test_lbl)}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 50)        500       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 50)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 50)        10050     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 50)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 2450)              0     

**Nhận xét**:
- Tổng số lượng params: **679,808**
- Model 1 có cấu trúc gồm 2 lớp Convolution và 4 lớp Dense (fully connected), tuy đã có 2 lớp Max Pooling nhưng tổng số lượng param vẫn khá nhiều so số lượng perceptrons trong lớp Dense cao. Về tổng quan thì model này tập trung nhiều vào các lớp Dense.
- Chính vì vậy, mô hình cho ra độ chính xác khá cao với xấp xỉ 97,6%, đồng thời tốc độ chạy nhanh (5 epochs và batch size 2000 chạy trong khoảng dưới 3 phút).

### **Mô hình 2 và nhận xét**

In [10]:
class Model2(AbstractModel):
    """Architecture for Model2.

    This model includes:
        - 1 Conv2D layer (32 filters, strides 3x3, ReLU activation)
        - 1 Conv2D layer (64 filters, strides 2x2, ReLU activation)
        - 2 BatchNormalization layers (used after each Conv2D layer)
        - 2 MaxPooling2D layers (size 2x2, used after each BatchNormalization layer)
        - 1 Flatten layer
        - 1 Dense layer (32 perceptrons, ReLU activation)
        - 1 Dense layer (10 perceptrons, Softmax activation)

    """

    def __init__(self, kernel_height: int, kernel_width: int) -> None:
        """Initialize layers in the model."""
        self.input_layer = Input(shape=(kernel_height, kernel_width, 1))
        self.conv_layer = Conv2D(32, (3, 3), activation="relu", padding="same")(self.input_layer)
        self.batch_layer = BatchNormalization()(self.conv_layer)
        self.pooling_layer = MaxPooling2D((2, 2))(self.batch_layer)
        self.conv_layer_2 = Conv2D(64, (2, 2), activation="relu", padding="same")(self.pooling_layer)
        self.batch_layer_2 = BatchNormalization()(self.conv_layer_2)
        self.pooling_layer_2 = MaxPooling2D((2, 2))(self.batch_layer_2)
        self.flatten_layer = Flatten()(self.pooling_layer_2)
        self.dense_layer = Dense(32, activation="relu")(self.flatten_layer)
        self.output_layer = Dense(10, activation="softmax")(self.dense_layer)
        self.model = Model(inputs=self.input_layer, outputs=self.output_layer)

In [11]:
model2 = Model2(28, 28)
model2.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)
model2.fit(train_img, train_lbl, test_img, test_lbl, 10, 1000)
print(f"Model Accuracy: {model2.accuracy(test_img, test_lbl)}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        8256      
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 64)       256 

**Nhận xét**:
- Tổng số lượng params: **109,482**
- Model 2 tập trung vào việc tăng số filter (32 và 64) trong lớp Convolution, giảm bớt các lớp Dense. Ngoài ra, model còn làm cân bằng các trọng số và tăng tốc độ training bằng 2 lớp Batch Normalization sau mỗi lớp tích chập. Chính vì vậy mà số lượng param của model 2 ít hơn nhiều so với model 1.
- Mô hình cho ra độ chính xác cao hơn Model 1 với xấp xỉ 98,7% accuracy (tập train có độ chính xác hơn 99%), tốc độ training cũng ngang với model 1 (10 epochs và batch size 1000 trong khoảng hơn 7 phút).
- Chung quy lại, mô hình 2 có hiệu suất rất tốt khi chỉ có số lượng param không quá nhiều nhưng đạt được độ chính xác cao, đồng thời tốc độ training cũng nhanh nhờ vào số param ít.

### **Mô hình 3 và nhận xét**

In [12]:
class Model3(AbstractModel):
    """Architecture for Model3.

    This model includes:
        - 1 Conv2D layer (64 filters, strides 3x3, ReLU activation)
        - 1 Conv2D layer (128 filters, strides 3x3, ReLU activation)
        - 1 SeparableConv2D layer (256 filters, strides 2x2, ReLU activation)
        - 3 BatchNormalization layers (used after each Conv2D/SeparableConv2D layer)
        - 2 MaxPooling2D layers (size 2x2, used after each BatchNormalization layer)
        - 1 Flatten layer
        - 1 Dense layer (128 perceptrons, ReLU activation)
        - 1 Dense layer (64 perceptrons, ReLU activation)
        - 1 Dense layer (32 perceptrons, ReLU activation)
        - 1 Dense layer (10 perceptrons, Softmax activation)

    """

    def __init__(self, kernel_height: int, kernel_width: int) -> None:
        """Initialize layers in the model."""
        self.input_layer = Input(shape=(kernel_height, kernel_width, 1))
        self.conv_layer = Conv2D(64, (3, 3), activation="relu", padding="same")(self.input_layer)
        self.batch_layer = BatchNormalization()(self.conv_layer)
        self.pooling_layer = MaxPooling2D((2, 2))(self.batch_layer)
        self.conv_layer_2 = Conv2D(128, (3, 3), activation="relu", padding="same")(self.pooling_layer)
        self.batch_layer_2 = BatchNormalization()(self.conv_layer_2)
        self.pooling_layer_2 = MaxPooling2D((2, 2))(self.batch_layer_2)
        self.sepconv_layer = SeparableConv2D(256, (2, 2), activation="relu", padding="same")(self.pooling_layer_2)
        self.batch_layer_3 = BatchNormalization()(self.sepconv_layer)
        self.flatten_layer = Flatten()(self.batch_layer_3)
        self.dense_layer = Dense(128, activation="relu")(self.flatten_layer)
        self.dense_layer_2 = Dense(64, activation="relu")(self.dense_layer)
        self.dense_layer_3 = Dense(32, activation="relu")(self.dense_layer_2)
        self.output_layer = Dense(10, activation="softmax")(self.dense_layer_3)
        self.model = Model(inputs=self.input_layer, outputs=self.output_layer)

In [13]:
model3 = Model3(28, 28)
model3.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)
model3.fit(train_img, train_lbl, test_img, test_lbl, 10, 2000)
print(f"Model Accuracy: {model3.accuracy(test_img, test_lbl)}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 64)        640       
                                                                 
 batch_normalization_2 (Batc  (None, 28, 28, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 batch_normalization_3 (Batc  (None, 14, 14, 128)      512 

**Nhận xét**:
- Tổng số lượng params: **1,725,354**
- Ở mô hình 3, số lượng layer được thiết lập dày đặc nhất, với 4 lớp Dense, lần lượt có 128, 64, 32 và 10 perceptrons, đồng thời có 2 lớp Convolution với số lớp filter cao (64 và 128 filter), 2 lớp Batch Normalization để cân bằng trọng số sau mỗi lớp Convolution.
- Ngoài ra, mô hình 3 còn được gắn thêm 1 lớp Separable Convolution để giảm bớt số lượng param mà vẫn cho ra cùng độ accuracy, tuy nhiên do quá nhiều lớp chồng lên nhau mà tổng số lương param nhiều nhất trong cả 3 mô hình.
- Mô hình 3 chạy chậm nhất (10 epoch, 2000 batch size nhưng thời gian training hơn 20 phút), bù lại cho ra độ chính xác cao nhất với 99% accuracy, trong đó accuracy trên tập train là gần như 100%.
- Tóm lại, ta có thể thấy được việc tăng số layer cho mô hình, đặc biệt là số lớp Dense sẽ làm cho mô hình có độ chính xác cao hơn, nhưng đánh đổi lại là tốc độ training lâu hơn khá nhiều.

### **Bảng so sánh**

| **Model** | **Params** | **Accuracy (%)** | **Avg Time** |            **Note**           |
|:---------:|:----------:|------------------|:--------:|:-----------------------------:|
|   Model1  |   679,808  |     ~97,6     |  2m 42s (5 epochs)  | Fastest time, Lowest accuracy |
|   Model2  |   109,482  |     ~98,7     |  7m 22s (10 epochs) | Least parameters, Best effiency (Time & Accuracy) |
|   Model3  |  1,725,354 |     ~99,0     |  26m 10s (10 epochs) | Lowest time, Highest accuracy |